In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"ny_od_main_JT00_2010.csv")

In [3]:
df.drop(columns=['createdate'], inplace=True)
df.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
0,360010001001004,360010001001009,1,0,1,0,0,0,1,1,0,0
1,360010001001004,360010001001010,1,0,1,0,0,0,1,1,0,0
2,360010001001004,360010001001033,1,0,1,0,0,0,1,1,0,0
3,360010001001004,360010014004001,1,0,1,0,0,0,1,0,1,0
4,360010001001004,360010015001007,1,0,1,0,0,0,1,0,1,0


In [4]:
df['w_tract'] = df['w_geocode'].astype(str).str[:11].astype(int)
df['h_tract'] = df['h_geocode'].astype(str).str[:11].astype(int)

w_geocode: workplace census tract code

h_geocode: home census tract code

S000: total number of jobs

SA01: number of workers age 29 or younger

SA02: number of workers age 30-54

SA03: number of workers age 55 or older

SE01: number of jobs with earnings 1250 or less

SE02: number of jobs with earnings 1251-3333

SE03: number of jobs with earnings above 3334

SI01: number of jobs in goods producing industry sectors

SI02: number of jobs in Trade, Transportation and Utilities

SI03: number of jobs in All Other Services

In [5]:
df_agg = df.groupby(['w_tract', 'h_tract']).sum().reset_index().drop(columns=['w_geocode', 'h_geocode'])

Central county in Albany OMB defined MSA: Albany County, Rensselaer County, Saratoga County, Schenectady County

Tracts in Albany County, Rensselaer County, Saratoga County, Schenectady County:
* Albany county: 36001
* Rensselaer county: 36083
* Saratoga county: 36091
* Schenectady county: 36093

Central county in Syracuse OMB defined MSA: 
* Onondaga County 36067

In [6]:
df_agg['w_county'] = df_agg['w_tract'].astype(str).str[:5].astype(int)
df_agg['h_county'] = df_agg['h_tract'].astype(str).str[:5].astype(int)
df_agg.head()

,w_tract,h_tract,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_county,h_county
0,36001000100,36001000100,29,5,18,6,4,10,15,5,15,9,36001,36001
1,36001000100,36001000201,11,2,7,2,3,6,2,0,8,3,36001,36001
2,36001000100,36001000202,5,1,4,0,0,5,0,0,4,1,36001,36001
3,36001000100,36001000301,3,2,1,0,0,2,1,0,1,2,36001,36001
4,36001000100,36001000302,10,3,6,1,2,7,1,1,3,6,36001,36001


In [7]:
df_agg['w_albany_core'] = df_agg['w_county'].apply(lambda x: 1 if x in [36001] else 0) #, 36083, 36091, 36093] else 0)
df_agg['h_albany_core'] = df_agg['h_county'].apply(lambda x: 1 if x in [36001] else 0) #, 36083, 36091, 36093] else 0)
df_agg['w_syracuse_core'] = df_agg['w_county'].apply(lambda x: 1 if x in [36067] else 0)
df_agg['h_syracuse_core'] = df_agg['h_county'].apply(lambda x: 1 if x in [36067] else 0)

In [8]:
albany_w_tracts = df_agg.query('w_albany_core==1')['w_tract'].unique()
albany_h_tracts = df_agg.query('h_albany_core==1')['h_tract'].unique()

In [9]:
#sanity check
assert len(albany_h_tracts) == len(albany_w_tracts), "AssertionError: the number of w and h tracts don't match"
print(len(albany_h_tracts))

85


In [10]:
syracuse_w_tracts = df_agg.query('w_syracuse_core==1')['w_tract'].unique()
syracuse_h_tracts = df_agg.query('h_syracuse_core==1')['h_tract'].unique()

In [11]:
#sanity check
assert len(syracuse_h_tracts) == len(syracuse_w_tracts), "AssertionError: the number of w and h tracts don't match"
print(len(syracuse_w_tracts))

142


In [12]:
df_agg_s = df_agg[['w_tract', 'h_tract', 'S000', 'w_albany_core', 'h_albany_core', 'w_syracuse_core', 'h_syracuse_core', 'w_county', 'h_county']]

In [14]:
def analysis_for_tract(tract_number):
    df_tract_w = df_agg_s.query(f'w_tract=={tract_number}')[['S000', 'h_county']].groupby(['h_county']).sum().reset_index()
    df_tract_sum_w = df_tract_w['S000'].sum()
    df_tract_w['share'] = df_tract_w['S000'] / df_tract_sum_w

    df_tract_h = df_agg_s.query(f'h_tract=={tract_number}')[['S000', 'w_county']].groupby(['w_county']).sum().reset_index()
    df_tract_sum_h = df_tract_h['S000'].sum()
    df_tract_h['share'] = df_tract_h['S000'] / df_tract_sum_h

    return df_tract_w.query('share>0.25'), df_tract_h.query('share>0.25')

In [ ]:
analysis_for_tract(36083052603) #we see that we will be able to attach this tract to the Albany core, given that 26% of people who live in it work in county 36001

(   h_county  S000     share
 6     36083   120  0.805369,
     w_county  S000     share
 0      36001   250  0.263158
 30     36083   310  0.326316)

In [16]:
total_worker_per_w_tract = df_agg_s[['w_tract', 'S000']].groupby('w_tract').sum().reset_index()
total_worker_per_h_tract = df_agg_s[['h_tract', 'S000']].groupby('h_tract').sum().reset_index()

In [17]:
df_agg_s1 = pd.merge(df_agg_s, total_worker_per_w_tract, left_on='w_tract', right_on='w_tract', how='inner')
df_agg_s2 = pd.merge(df_agg_s1, total_worker_per_h_tract, left_on='h_tract', right_on='h_tract', how='inner')

In [18]:
df_agg_s2.rename(columns={'S000_x': 'S000', 'S000_y': 'S000_w_total', 'S000': 'S000_h_total'}, inplace=True)

In [20]:
flow_matrix = df_agg_s2.pivot(index='h_tract', 
                              columns='w_tract', 
                              values='S000').fillna(0) #convert to a (#home x #work) matrix

In [21]:
tract_home_rows = flow_matrix.index 
tract_work_cols = flow_matrix.columns
flow_matrix_np = flow_matrix.astype(int).values #each row is a home, each column is a work
total_home = np.sum(flow_matrix_np, axis=1)
total_work = np.sum(flow_matrix_np, axis=0)

In [22]:
counties_home_rows = np.array(tract_home_rows.astype(str).str[:5]).astype(int)
counties_work_cols = np.array(tract_work_cols.astype(str).str[:5]).astype(int)

In [104]:
core_albany_init = np.unique(np.concatenate((tract_home_rows[counties_home_rows==36001], tract_work_cols[counties_work_cols==36001]), axis=0))
core_syracuse_init = np.unique(np.concatenate((tract_home_rows[counties_home_rows==36067], tract_work_cols[counties_work_cols==36067]), axis=0))

In [105]:
def update_core(core, flow_matrix, tract_home_rows, tract_work_cols):
    """
    This function updates the core, by adding the tracts that are not in the core but have either:
    * >25% of people who live in the tract work in the core
    * >25% of people who work in the tract live in the core
    """
    row_index_core = np.where(np.isin(tract_home_rows, core))[0]
    col_index_core = np.where(np.isin(tract_work_cols, core))[0]
    total_home = np.sum(flow_matrix, axis=1)
    total_work = np.sum(flow_matrix, axis=0)

    # find indexes of rows and columns that are not in the core but fit the definition 
    indexes_rows_add_core = np.setdiff1d(np.where((flow_matrix[:, col_index_core].sum(axis=1)/total_home) > 0.26)[0], row_index_core, assume_unique=True)
    indexes_cols_add_core = np.setdiff1d(np.where((flow_matrix[row_index_core, :].sum(axis=0)/total_work) > 0.26)[0], col_index_core, assume_unique=True)

    # adds these new tracts to the core
    new_core = np.union1d(core, np.concatenate((tract_home_rows[indexes_rows_add_core], tract_work_cols[indexes_cols_add_core]), axis=0))
    return np.unique(new_core)

In [110]:
def update_core_iterative(core, flow_matrix, tract_home_rows, tract_work_cols):
    """
    This function updates the core iteratively until no new tracts are added
    """
    core_init_length = core.shape[0]
    while True:
        new_core = update_core(core, flow_matrix, tract_home_rows, tract_work_cols)
        if np.array_equal(new_core, core):
            break
        core = new_core
    core_final_length = core.shape[0]
    print(f"Number of tracts added: {core_final_length - core_init_length}")
    return core

In [111]:
core_albany = update_core_iterative(core_albany_init, flow_matrix_np, tract_home_rows, tract_work_cols)

Number of tracts added: 299


In [112]:
core_syracuse = update_core_iterative(core_syracuse_init, flow_matrix_np, tract_home_rows, tract_work_cols)

Number of tracts added: 101
